In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os

# Add the parent directory to the path
sys.path.append(os.path.join(os.getcwd(), ".."))

from calculate.calc import get_site_dfs, load_dfs

import polars as pl
from polars import col

In [ ]:
(
    joinyears,
    df_users,
    df_users_registered,
    df_posts_all,
    df_comments_all,
    df_activity_all,
) = load_dfs("../infodump")

df_months, df_posts, df_comments, df_activity = get_site_dfs(
    "askme", df_posts_all, df_comments_all, df_activity_all
)

In [ ]:
df_posts

In [ ]:
df_human_relations = (
    df_months.join(
        df_posts.select("month", "category"), on="month", how="left", coalesce=True
    )
    .group_by("month")
    .agg(pl.len(), (col("category") == 14).sum().alias("hr"))
    .with_columns(hr_pc=col("hr") / col("len"))
    .with_columns(
        rolling_hr=col("hr").rolling_mean_by("month", window_size="12mo").round(0),
        rolling_hr_pc=col("hr_pc").rolling_mean_by("month", window_size="12mo"),
    )
)

df_human_relations

In [ ]:
chart = (
    df_human_relations.plot.line(
        x="month",
        y="rolling_hr_pc",
    )
    .properties(
        title="Proportion of Human Relations questions on Ask MetaFilter (12-month rolling average)",
    )
    .configure_axisY(format="%")
)
chart
